In [120]:
#Import all required libraries

from bs4 import BeautifulSoup
import requests
import pandas as pd
from IPython.display import HTML
import numpy as np

In [94]:
# url of the website to scrape
url = 'https://www.imdb.com/chart/top/'

In [95]:
# making request to that website using url
r = requests.get(url)

In [96]:
# getting content of the website
body = r.text

In [97]:
# passing content of the website to BeautifulSoup to parse as HTML page
soup = BeautifulSoup(body, 'html')

In [98]:
# find all table that have following styled class
table = soup.find_all('table', class_='chart full-width')

In [99]:
# get the first table from the result
table = table[0]

In [100]:
# finding and iterating the table to save name of the movie
names = []
for row in table.find_all('tr'):
    for cell in row.find_all('td', class_='titleColumn'):
        for link in cell.find_all('a'):
            names.append(link.text)

In [101]:
# finding and iterating the table to save rating of the movie
rating = []
for row in table.find_all('tr'):
    for cell in row.find_all('td', class_='imdbRating'):
        for stars in cell.find_all('strong'):
            rating.append(stars.text)

In [102]:
# finding and iterating the table to find link for each movie
links = []
for row in table.find_all('tr'):
    for cell in row.find_all('td', class_='titleColumn'):
        for link in cell.find_all('a'):
            whole_link = 'https://www.imdb.com' + link.get('href')
            links.append(whole_link)

In [103]:
# finding and iterating the table to find poster link of the movies
poster_link = []
for row in table.find_all('tr'):
    for cell in row.find_all('td', class_='posterColumn'):
        for link in cell.find_all('img'):
            poster_link.append(link.get('src'))

In [ ]:
# finding and iterating the table to find released year and small description of the movie
summary = []
year_data = []
index = 0

for l in links:
    r = requests.get(l)
    body = r.text
    soup = BeautifulSoup(body, 'html')
    for heading in soup.find_all('div', class_='summary_text'):
        result = heading.text
        newstr = "".join(result.split('\n'))
        summary.append(newstr)
    for year in soup.find_all('div', class_='title_wrapper'):
        for span in year.find_all('h1', class_=''):
            yr = span.text
            yr = yr.split()
            yr = yr[-1].replace('(', "").replace(')', "")
            year_data.append(yr)
            index = index + 1

In [89]:
# Creating df from the list of datas and set to specific column to store 
df = pd.DataFrame(list(zip(year_data, names, rating, links, summary, poster_link)), 
                  columns=['Released Year', 'Name', 'Rating', 'Link', 'Summary', 'Poster link'])

In [123]:
# Set start index value to one
df.index = np.arange(1, len(df) + 1)

# Checking first five rows of data
df.head()

,Year,Name,Rating,Link,Summary,Poster link
1,1994,The Shawshank Redemption,9.2,https://www.imdb.com/title/tt0111161/,Two imprisoned men bond ov...,https://m.media-amazon.com/images/M/MV5BMDFkYT...
2,1972,The Godfather,9.1,https://www.imdb.com/title/tt0068646/,The aging patriarch of an ...,https://m.media-amazon.com/images/M/MV5BM2MyNj...
3,1974,The Godfather: Part II,9.0,https://www.imdb.com/title/tt0071562/,The early life and career ...,https://m.media-amazon.com/images/M/MV5BMWMwMG...
4,2008,The Dark Knight,9.0,https://www.imdb.com/title/tt0468569/,When the menace known as t...,https://m.media-amazon.com/images/M/MV5BMTMxNT...
5,1957,12 Angry Men,8.9,https://www.imdb.com/title/tt0050083/,A jury holdout attempts to...,https://m.media-amazon.com/images/M/MV5BMWU4N2...


In [ ]:
# saving df as HTML table
#HTML(df.to_html())

In [103]:
# saving df as csv file
df.to_csv('Top Rated Movies.csv', index=True)

In [104]:
# saving df as excel file
df.to_excel('Top Rated Movies.xlsx', index=False, header=True)